In [1]:
# default_exp Web

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
#export
import os
import sys
import shutil
import json
from jinja2 import Environment, FileSystemLoader, Template

# Web functions 

> API details.

In [4]:
#hide
from nbdev.showdoc import *

cPath = os.path.dirname(os.getcwd())
cPath
sys.path.insert(0,cPath)

In [21]:
#export
from refreshment.School import StudySystem
from refreshment.Program import Program, Subject, Record, Lesson
from refreshment.munge import  dateGrid, processGuide, unassignedForSubject, printAllUnassigned

In [6]:
#hide
guide = StudySystem("./data/programs.json")
for x in guide.programs:
    print(x.name)
    for y in x.subjects:
        print("\"" + x.name + "\",\"" + y.name + "\"")

In [7]:
guide.loadDirectory()
for x in guide.programs:
    print(x.name)
    for y in x.subjects:
        print("\"" + x.name + "\",\"" + y.name + "\"")

Cathedral
"Cathedral","Reading"
"Cathedral","Music"
"Cathedral","Chapel"
"Cathedral","Science"
"Cathedral","HealthAndExercise"
"Cathedral","Schedules"
"Cathedral","Math"
"Cathedral","Mindful"
"Cathedral","EmotionalLearning"
"Cathedral","Conners"
"Cathedral","Writing"
web
"web","Cathedral"
Lemonade
"Lemonade","Lemonade"
"Lemonade","Lemonade.xcodeproj"
"Lemonade","LemonadeTests"
"Lemonade",".git"


In [8]:
#hide
cath = guide.programs[0]
foo = cath.subjects
sorted(foo, key=lambda x: x.name)[:2]

[Chapel; [CSB Lower School Distance Chapel March 31th, 2020.mp4|1|1|CSB Lower School Distance Chapel March 31th, 2020.mp4,CSB Lower School Distance Chapel April 7th, 2020.mp4|2|2|CSB Lower School Distance Chapel April 7th, 2020.mp4,CSB Lower School Distance Chapel April 21st, 2020.mp4|3|3|CSB Lower School Distance Chapel April 21st, 2020.mp4,CSB Lower School Distance Chapel April 28th, 2020.mp4|4|4|CSB Lower School Distance Chapel April 28th, 2020.mp4,CSB Lower School Distance Chapel May 5th, 2020.mp4|5|5|CSB Lower School Distance Chapel May 5th, 2020.mp4,Kindergarten Readiness - Lower School Chapel.mp4|6|6|Kindergarten Readiness - Lower School Chapel.mp4,CSB Lower School Distance Chapel May 12th, 2020.mp4|7|7|CSB Lower School Distance Chapel May 12th, 2020.mp4],
 Conners; [Edward C Class Code.pdf|1|1|Edward C Class Code.pdf,Kindergarten Family Directory 20-21.pdf|2|2|Kindergarten Family Directory 20-21.pdf,EdwardLetterConceptV.mp4|3|3|EdwardLetterConceptV.mp4,EdwardKingdomOfY.mp4|4|4|

In [9]:
reading = [x  for x in cath.subjects if x.name == "Reading"][0]
for less in reading.lessons:
    print(less.fileName)

SnapByFor.mp4
ThreePart.03.31.mp4
Fortunately.mp4
LittleBlueTruck.mp4
ThreePart.03.25.mp4
5SightWords.mp4
truestory.mp4
VisitMusem.mp4
MOWiliiams.mp4
ReadAloud.mp4
LetterConceptE.mp4
LetterEScavenegerHunt.mp4
ThreePart.4.7.20.mp4
AmosMcGee.mp4
WhatBooks.mp4
DavidTellsJoke.mp4
SnapWordsMemory..mp4
SnapWords.mp4
aChairForMyMother.mp4
FantasticElasticBrain.mp4
FrogToad.mp4
LetterConceptS.mp4
ThreePart.04.21.mp4
LetterConceptSH.mp4
HowToBabySitAGrandPa.mp4
ReadingFromHome.mp4
ThreePartDrill.4.23.20.mp4
LetterConceptTH.mp4
ThreePart.4.28.mp4
LetterConceptW.mp4
AvidReader.mp4
ThreePart.4.30.mp4
LetterConceptY.mp4
ThreePartDrill.05.06.mp4
BearsFeelScared.mp4
Reading-BookReview.pdf
BigHugsLittleHugs.mp4
LetterConceptV.mp4
ThreePartDrill.5.12.20.mp4
Review Writing 5_12.pdf
Find the rhymes.m4a
LetterConceptX.mp4
ThreePart.05.14.mp4
LetterConceptZ.mp4
FrogAndToad.mp4
SW Bingo .pdf


In [10]:
#hide
template = Template('Hello {{ name }}!')
template.render(name=cath.name)

'Hello Cathedral!'

In [11]:
#hide
guide = StudySystem("./data/programs.json")
for x in guide.programs:
    print(x.name)
    for y in x.subjects:
        print("\"" + x.name + "\",\"" + y.name + "\"")

In [12]:
#export

class Render:
    def __init__(self, program, guide , dir="../../web",template = "../templates"):
       
        self.guide = guide
        self.program = [x for x in guide.programs if x.name == program][0]
        print("loaded program " + self.program.name)
        self.outputdir = os.path.join(dir,self.program.name)
        self.template = template
        
        self.file_loader = FileSystemLoader(self.template)
        self.env = Environment(loader=self.file_loader)
        self.resources = "resources"

        
    def basePath(self):
        return self.outputdir
    
    def renderLesson(self,sub,lesson):
        template = self.env.get_template("lesson.html")
        resDir = "../../.."
        
        output = template.render(item=sub,
                                 resDir=resDir,
                                 less=lesson,
                                 program = self.program,
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"vid_" +str(lesson.key) + ".html"), "w")
        f.write(output)
        
    def renderSubject(self,sub):
        template = self.env.get_template("subject.html")
        videos = [x for x in sub.lessons if x.fileName.endswith(".mp4")] 
        resources = [x for x in sub.lessons if not x.fileName.endswith(".mp4")]
        for x in sub.lessons:
            self.renderLesson(sub,x)
        resDir = "../../.."
        #print(sub.sequences)
        output = template.render(item=sub,
                                 videos=videos,
                                 res=resources,
                                 resDir=resDir,
                                 program = self.program,
                                 grid = reversed(dateGrid(self.program,subject=sub.name)),
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"index.html"), "w")
        f.write(output)
       
    def renderCalendar(self):
        
        grid = reversed(dateGrid(self.program))

        if not os.path.exists(self.outputdir):
            os.mkdir(self.outputdir)
  
        template = self.env.get_template("calendar.html")
        output = template.render(program=self.program,
                                 grid=grid,
                                 styleDir=os.path.join(".",self.resources))
        
        f = open(os.path.join(self.outputdir,"calendar.html"), "w")
        f.write(output)
        f.close()
        
    def renderSchool(self):
        
        foo = self.program.subjects
        seq = sorted(foo, key=lambda x: x.name)

        if not os.path.exists(self.outputdir):
            os.mkdir(self.outputdir)
            
        for x in seq:
            self.renderSubject(x)
            
        template = self.env.get_template("school.html")
        output = template.render(program=self.program,
                                 seq=seq,
                                 styleDir=os.path.join(".",self.resources))
        
        f = open(os.path.join(self.outputdir,"index.html"), "w")
        f.write(output)
        f.close()
        
        self.addResources()
        
    def addResources(self):
        resDir = os.path.join(self.outputdir,self.resources)
        if not os.path.exists(resDir):
            os.mkdir(resDir)
        startDir = os.path.join(".", self.resources)
        src_files = os.listdir(startDir)
        
        for file_name in src_files:
            full_file_name = os.path.join(startDir, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, os.path.join(resDir, file_name))
        
        with open(os.path.join(resDir, "lemonade.json"), "w") as dataFile:
            json.dump(self.program.toDict(), dataFile, indent=4, sort_keys=True)
       
    def addFiles(self):
        for sub in self.program.subjects:
            for lesson in sub.lessons:
                source = os.path.join(self.guide.origin, self.program.name,sub.name,lesson.fileName)
                dest = os.path.join(self.outputdir,sub.name,lesson.fileName)
                
                if not os.path.isfile(dest):
                    print("found " + dest)
                    shutil.copy(source, dest)
        


In [23]:
#hide
guide.loadDirectory()
processGuide(guide)
cath = guide.programs[0]


In [24]:
printAllUnassigned(prog=cath)


Reading
['truestory.mp4', 'VisitMusem.mp4', 'LetterEScavenegerHunt.mp4', 'WhatBooks.mp4', 'SnapWords.mp4', 'FantasticElasticBrain.mp4', 'ReadingFromHome.mp4', 'AvidReader.mp4', 'Reading-BookReview.pdf', 'Review Writing 5_12.pdf', 'Find the rhymes.m4a', 'SW Bingo .pdf']

Emotional Learning
['Emotions.mp4', 'Fillbucket.mp4', 'Creative.mp4', 'Gratitude.mp4', 'Gratitude 422.mp4', 'Gratitude.4.21.mp4', 'SharingPicture.mp4', 'GratitudeTree.mp4', 'RainShadow.mp4']

Writing
['VirtualMuseum.mp4', 'EnjoyAboutDistanceLearning.mp4', 'FeelGood.mp4', 'GK_PC_BK_Box4Lines.pdf', 'imaging.seesaw.me.png']


In [25]:
#export
def makeSite(school,programName):
    baseRender = Render(programName,school)
    baseRender.renderSchool()
    school.save()
    baseRender.addFiles()
    baseRender.addResources()
    baseRender.renderCalendar()

In [26]:
#hide
makeSite(school=guide,programName="Cathedral")

loaded program Cathedral
found ../../web/Cathedral/EmotionalLearning/KimochiBellaRose.mp4


In [16]:
#import json
#print(json.dumps(guide.toDict(), sort_keys=True, indent=4)) 